<a href="https://colab.research.google.com/github/edoardorossi1995/Tesi/blob/main/Average_Reward_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract

Algoritmo Q-Learningc (off policy)

Task non episodico (continuativo)

Framework: Average Reward

Approssimatore funzionale: Rete Neurale Convessa *(cfr. articoli)*






#Drive Setting


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  #!ls
else:
  %cd /Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive

import warnings
import pickle
import sys
if IN_COLAB == True:
  sys.path.insert(0, '/content/gdrive/MyDrive/Tesi/functions')
  sys.path.insert(0, '/content/gdrive/MyDrive/Tesi')
else:
  sys.path.insert(0, '/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive/functions')
  sys.path.insert(0, '/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive')

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.layers import Activation
from keras.activations import exponential, relu
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import Layer, InputSpec, Reshape
from tensorflow.keras import initializers
from scipy.optimize import minimize

from pkl import store_data, load_data
from functions.compressor import compress

import random
from collections import defaultdict

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tesi


# Import Anagrafica e CTA

In [2]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
if IN_COLAB == True:
  CTA30_def = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_def.csv']
else:
  CTA30_def = ['/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive/CTA30/DF_clean/clean_CTA_30_def.csv']
  
df_def = pd.read_csv(CTA30_def[0])

if IN_COLAB == True:
  CTA30_18 = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_2018.csv']
else:
  CTA30_18 = ['/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive/CTA30/DF_clean/clean_CTA_30_2018.csv']

df = pd.read_csv(CTA30_18[0])
df['VLV_RAF_FDBK_00'] = df['VLV_RAF_FDBK_00'] / 100
df['VLV_SUR_FDBK_00'] = df['VLV_SUR_FDBK_00'] / 100
scaler = MinMaxScaler()
act_f_m = df.loc[:,'F_M_FDBK_00']
a_f_m_reshaped = np.array(act_f_m).reshape(-1, 1)
a_f_m_scaled = scaler.fit_transform(a_f_m_reshaped)

pca_df = compress(df)

In [3]:
print("Anagrafica shape:",df_def.shape,"\nDataframe shape:", df.shape,"\nPca Dataframe shape:", pca_df.shape)

Anagrafica shape: (76, 2) 
Dataframe shape: (141811, 76) 
Pca Dataframe shape: (141811, 7)


In [4]:
df_def

,pointColName,pointName
0,AUTOMAN_SERR_CMD_00,Automatico Manuale Serrande Comando
1,AUTOMAN_VLV_CMD_00,Automatico Manuale Valvole Comando
2,AUTOMAN_M_CMD_00,Automatico Manuale Mandata Comando
3,AUTOMAN_R_CMD_00,Automatico Manuale Ripresa Comando
4,LOCREM_CMD_00,Selettore Locale Remoto Comando
...,...,...
71,VOLT_R_00,Tensione Ripresa
72,T_M_SP_00,Temperatura Aria Mandata Setpoint
73,T_R_SP_00,Temperatura Aria Ritorno Setpoint
74,T_AMB_04,Temperatura Aria Ambiente Media


# Rete Neurale


## Custom Acts

Efficient reinforcement learning via difference of
log-sum-exp neural networks

(Sven Bru ̈ggemann, Corrado Possieri)

In [42]:
class ExpAct(Layer):
    def __init__(self, input_dim, inner_dim, **kwargs):
        self.inner_dim = inner_dim
        self.input_dim = input_dim
        super(ExpAct, self).__init__(**kwargs)
        self.T = 0.1
        self.a = self.add_weight(name="a", shape=(self.input_dim, self.inner_dim), initializer="glorot_normal")
        self.b = self.add_weight(name="b", shape=(inner_dim,), initializer="zeros")

    def build(self, input_shape):
        self.input_spec = InputSpec(shape=(None, self.input_dim))
        super(ExpAct, self).build(input_shape)
        print("self.input_dim = ", self.input_dim)

    def call(self, x):
        #x.shape = (None,self.input_dim)
        z = x / self.T
        PSI = 5#self.psi(x, self.a, self.b)
        a_t = K.transpose(self.a)
        # N.B. output è una matrice 32x1, quindi PSI dovrà rispettare la dimensionalità
        tPSI = tf.fill((self.inner_dim,1), PSI)
        print("psi shape",tPSI.shape)
        jj = K.dot(a_t, z)
        print("az shape", x.shape)
        output = (K.dot(a_t, z) + self.b) / self.T - PSI
        print("output dim", output.shape)
        return output

    def psi(self, x, a, b):
        z = x / self.T
        a_t = K.transpose(self.a)
        output = (K.dot(a_t, z) + self.b) / self.T
        psi = max(output)
        return 5
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.inner_dim)

class LogAct(Layer):
    def __init__(self, inner_dim, **kwargs):
        super(LogAct, self).__init__(**kwargs)
        self.T = 0.1
        self.a = self.add_weight(name="a", shape=(input_dim, inner_dim), initializer="glorot_normal")
        self.b = self.add_weight(name="b", shape=(inner_dim,), initializer="zeros")
        self.inner_dim = inner_dim

    def call(self, x):
        z = x / self.T
        PSI = self.psi(x, self.a, self.b)
        p = self.T * (PSI + K.log(x))
        return p

    def psi(self, x, a, b):
        z = x / self.T    
        z = tf.transpose(tf.expand_dims(x, axis=1), perm=[0, 2, 1])
        PSI = K.dot(a,z) / self.T + b
        PSI = K.exp(PSI - K.max(PSI))
        PSI /= K.sum(PSI, axis=-1, keepdims=True)
        PSI *= self.inner_dim
        return PSI

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.inner_dim)

## NN Creation

Log-sum-exp neural networks and posynomial
models for convex and log-log-convex data 

(Calafiore, Gaubert, Possieri)

In [43]:
n = load_data('*.pkl')
action_vector_dimension = 2

# Architettura rete neurale
input_dim = n['n_components'] + action_vector_dimension
inner_dim_1 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer dim: ", inner_dim_1)
print("output dim: ", output_dim)

# Costruzione strati
rete_neurale = Sequential()

# Primo strato
rete_neurale.add(Dense(inner_dim_1, input_shape=(9,32), activation=ExpAct(input_dim=input_dim, inner_dim=inner_dim_1 )))
#rete_neurale.add(Dense(inner_dim_1, activation=ExpAct(inner_dim=inner_dim_1, input_dim=input_dim)))

# Secondo strato
rete_neurale.add(Dense(output_dim, activation=LogAct(inner_dim=output_dim)))

rete_neurale.build(input_shape=(None, input_dim))

rete_neurale.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

# Definisco funzione obiettivo per la massimizzazione della predizione 

def objective(x):
    pca_s_prime = pca_df.loc[s_prime,:]
    nn_input_prime = np.concatenate((pca_s_prime.values.reshape(7,1), x[0].reshape(1,1), x[1].reshape(1,1)), axis=0)
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))
    print(nn_input_prime)
    funct = rete_neurale.predict(nn_input_prime, verbose=0)[0][0]
    print("3")
    return -funct

input dim:  9
inner layer dim:  32
output dim:  1
self.input_dim =  9
psi shape (32, 1)
az shape (None, 9, 32)
output dim (32, None, 32)


ValueError: ignored

In [15]:
rete_neurale.input_shape

(None, 9)

In [ ]:
(df.loc[5000:6000,'PW_ATT_M_00'])

5000    19.8775
5001    19.7422
5002    19.7584
5003    19.7964
5004    19.8194
         ...   
5996    20.0913
5997    20.0210
5998    20.0126
5999    19.9825
6000    20.0735
Name: PW_ATT_M_00, Length: 1001, dtype: float64

# Q-Learning Agent

In [ ]:
# Dimensione problema e inizializzazione stato iniziale
[S, feat] = df.shape
current_state = 0 # np.random.randint(0,S-1)

# Stato corrente
s = current_state

min_f = np.min(a_f_m_scaled)
max_f = np.max(a_f_m_scaled)

# Parametri di apprendimento (learning rate, discount factor, average rate)
alpha = 0.5 # non in uso
gamma = 1
beta = 1

# Media mobile esponenziale (avr = avr + beta * (r - avr)) 
# [scelta perché computazionalmente più leggera di una media aritmetica]
average_reward = -np.random.randint(np.min(df.loc[:,'PW_ATT_M_00']),np.max(df.loc[:,'PW_ATT_M_00']))
print("average_reward start: ", average_reward,"\n")

# Setting Episodi
total_episodes = 50000
current_episode = 0

# Ripeti finché non raggiungi la fine del processo
for current_episode in range(total_episodes):
    
    # Stato successivo e ottieni l'azione A e la ricompensa R
    s_prime = s+1
    if s_prime > S-1:
      print("s =", s)
      break

    a1 = df.loc[s,'F_M_FDBK_00']
    a2 = df.loc[s,'VLV_RAF_FDBK_00']
    a = (a1,a2)
    r = -df.loc[s,'PW_ATT_M_00']
    average_reward = average_reward + beta * (r - average_reward)

    # Condizioni iniziali (random) della minimizzazione della funzione obiettivo 
    # per la predizione di q'
    data_f = a_f_m_scaled
    a0_f = data_f[np.random.randint(len(data_f))]
    a0_v = np.random.rand()
    x0 = [a0_f, a0_v]

    # Dominio della minimizzazione 
    bounds = [(min_f, max_f), (0,1)]

    # Risultato della minimizzazione
    objective_minimized = minimize(objective, x0, method = 'BFGS', bounds = bounds)

    # Compongo input della predizione di q'
    a1_prime_input = objective_minimized.x[0].reshape(1,1)
    a2_prime_input = objective_minimized.x[1].reshape(1,1)
    s_prime_input = pca_df.loc[s_prime,:].values.reshape(7,1)
    nn_input_prime = np.concatenate((s_prime_input, a1_prime_input, a2_prime_input),axis = 0)
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))

    # Predizione di q'
    q_prime = rete_neurale.predict(nn_input_prime, verbose = 0)[0]
    q_prime = q_prime[0]
    # Calcola la media delle ricompense future attese per lo stato-azione corrente
    # q = avr + gamma*max(q')
    #print("q' = ", q_prime)
    q = (average_reward + gamma * (q_prime))
    #print("q = ", q)

    # Calcola il valore target per il training della rete neurale
    target = q
    target = np.array([target])

    # Aggiorna la rete neurale
    pca_s = pca_df.loc[s,:]
    nn_input = np.concatenate((pca_s.values, np.array([a[0],a[1]])))
    nn_input = np.reshape(nn_input, (1,input_dim))    
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiorna lo stato corrente
    s = s_prime

    # Plot statistiche ogni 100 episodi
    print("current_episode = ", current_episode)
    n_print = 1
    if current_episode%n_print == 0:
      print("Episodio ",current_episode,"/",total_episodes)
      print("target: ", target, "\nreward: ",r,"\naverage_reward: ",np.mean(average_reward))
    print("\n")

# Restituisci la rete neurale ottenuta
#return rete_neurale

In [ ]:
rete_neurale.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (9, 9)                    180       
                                                                 
 dense_5 (Dense)             (9, 9, 1)                 20        
                                                                 
Total params: 200
Trainable params: 200
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rete_neurale.input_shape

(None, 9)